<p style="text-align:right">
    项目名称：<b>人脸识别</b><br/>
设计：董相志<br/>
学号：220100<br/>
日期：2020.10<br/></p>

# 第四阶段：模仿刷脸门禁系统实时人脸识别
基于<b>One-Shot Learning</b>（单样本学习）实现人脸实时检测与识别。<br/>
### 设计摘要:
【1】 将本单位的员工照片放入数据库，每位员工只有一张照片。本案例将员工照片存放于 "./employee"目录中。<br/>
【2】 用 haarcascade 方法或者 dlib 方法或者mtcnn方法，从摄像头实时捕获人脸图像。<br/>
【3】 用训练好的模型VGGFace Model分别提取员工的实时照片 与 数据库照片 的特征。<font color ='red'>其实更好的做法是将员工照片的特征单独存储。</font><br/>
【4】 用余弦相似性将摄像头捕获的图像与员工数据库比对，匹配，开闸放行；否则，拒绝进入。<br/>


## 4.1 制作员工照片数据库

用下面的程序段采集几张员工照片放到employee目录。

In [ ]:
# 导入库
import cv2   
import dlib
import numpy as np

# 定义面部正面探测器
detector = dlib.get_frontal_face_detector()

# 打开摄像头或者打开视频文件
cap = cv2.VideoCapture(0)  #参数设为0，可以从摄像头实时采集头像
frame_count = 0  #帧计数
face_count = 0  #脸部计数

# 循环读取每一帧，对每一帧做脸部检测，按ESC键循环结束
while True:
    key = cv2.waitKey(1) & 0xFF   # 读键盘
    
    ret, frame = cap.read()  #从摄像头或者文件中读取一帧
    if (ret != True):
        print('没有捕获图像，数据采集结束或者检查摄像头是否工作正常！')
        break 
        
    frame_count += 1
 
    img_h, img_w, _ = np.shape(frame)  # 获取图像尺寸
    
    detected = detector(frame, 1)  #对当前帧检测
    faces = []   # 脸部图像列表
    
    if len(detected) > 0:  #当前帧检测到脸部
        for i, d in enumerate(detected):
            
            # 脸部图像坐标与尺寸
            x1, y1, x2, y2, w, h = d.left(), d.top(), \
                                   d.right() + 1, d.bottom() + 1, \
                                   d.width(), d.height()
            
            # 脸部图像坐标
            face =  frame[y1:y2 + 1, x1:x2 + 1, :]
            face = cv2.resize(face, (128, 128), interpolation = cv2.INTER_CUBIC)
            if (key == 32): #空格键采集头像
                face_count += 1
                file_name = "./employee/"+str(frame_count)+str(i)+".jpg"
                cv2.imwrite(file_name, face)
                print('员工照片已经保存到employee目录！！')
            elif (key == 27): #ESC键退出
                break

            # 绘制边界框
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # 显示单帧检测结果
    cv2.imshow("Face Detector", frame) 
    # Esc键终止检测
    if key == 27:
        break
print('已经完成了 {0} 帧检测，保存了 {1} 幅脸部图像'.format(frame_count, face_count))
cap.release()
cv2.destroyAllWindows()      


摆好姿势，按空格键就像按下快门键一样，照片采集到employee目录。可以让同学轮流站到摄像头前，每到这个时候，感觉总是放松好玩的。按下ESC键结束员工照片采集。这些照片将用于人脸实时识别的依据。<br/>
接下来，可以检验实战效果了。

## 4.2 加载预训练模型 VGGFace Model 


当然需要首先加载预训练模型，这个工作在第三阶段做了简化处理。

In [1]:
# 导入库
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Convolution2D, MaxPool2D, Dense, \
     ZeroPadding2D, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 按照论文中参数设定，定义VGGFace模型
model = Sequential(name = 'VGGFace-Model')
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))  # 输入层
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPool2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPool2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPool2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPool2D((2,2), strides=(2,2)))

model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))
          
# 模型结构
model.summary()

Model: "VGGFace-Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)   

In [3]:
model.load_weights('model/vgg_face_weights.h5')

In [4]:
# 定义特征提取模型，舍去最后一层(即Softmax激活函数层)
face_model = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

## 4.3 基于余弦距离对摄像头捕获的人脸实时检测识别


首先对员工照片做特征提取，存入特征字典。实践中特征是存入数据库的。

In [14]:
import cv2
import dlib
import numpy as np
from os import listdir
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import preprocess_input

In [15]:
%%time
employee_dir = "./employee/"   #员工照片目录
all_people_faces = dict()  # 员工照片特征字典，临时存储
face_h, face_w = 224,224  #模型输入的头像大小

# 读取图像并做预处理，用于对员工目录的照片做预处理
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

for file in listdir(employee_dir):  # 遍历员工目录
    person, extension = file.split(".")
    # 析取特征加入字典
    all_people_faces[person] = face_model.predict( \
                               preprocess_image(f'./employee/{person}.jpg'))[0,:]

print(f"成功提取所有员工:{all_people_faces.keys()}的特征到数据字典！！")

成功提取所有员工:dict_keys(['120', '270', '290', '350', 'dongxiangzhi', 'yangshuang'])的特征到数据字典！！
Wall time: 496 ms


抽样显示某一位员工的脸部特征编码如下：

In [16]:
all_people_faces['dongxiangzhi']

array([ 0.32678002, -1.2226102 , -0.51887906, ..., -2.1883993 ,
        1.5485346 ,  0.8096776 ], dtype=float32)

查看特征编码长度：

In [17]:
len(all_people_faces['dongxiangzhi'])

2622

In [18]:
print(all_people_faces['dongxiangzhi'].shape)

(2622,)


2622,这正是VGGFace的输出维度。有别于FaceNet的128、512维度。

In [19]:
# 计算余弦距离的函数
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1-(a / (np.sqrt(b) * np.sqrt(c)))

In [20]:
# 计算欧氏距离
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

下面的程序段完成实时检测识别。

In [22]:
thresh = 0.4  #设定余弦距离阈值，低于这个值，认为是同一个人

# 定义面部正面探测器
detector = dlib.get_frontal_face_detector()

#  打开摄像头
cap = cv2.VideoCapture(0) 

while(cap.isOpened()):
    ret, frame = cap.read()  #读取一帧
    
    frame_h, frame_w, _ = np.shape(frame)  # 帧图像大小
    detected = detector(frame, 1)  #对当前帧检测
    if len(detected) > 0:  #提取当前帧探测的所有脸部图像，构建预测数据集
        for i, d in enumerate(detected):  #枚举脸部对象
            #脸部坐标
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            # 绘制边界框
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            # 脸部的边界
            face =  frame[y1:y2 + 1, x1:x2 + 1, :]
            # 脸部缩放，以适合模型需要的输入维度
            face = cv2.resize(face, (face_h, face_w))
            # 图像归一化
            face = face.astype("float") / 255.0
            # 扩充维度，变为四维（1，face_h,face_w,3）
            face = np.expand_dims(face, axis=0)
                        
            # 用模型进行特征提取，这是长度为2622的特征向量
            captured_representation = face_model.predict(face)[0,:]
            print(captured_representation.shape)
            # 到员工数据库比对
            found = 0
            for i in all_people_faces:
                person_name = i
                representation = all_people_faces[i]

                similarity = findCosineSimilarity(representation, \
                             captured_representation)
                print('余弦距离',similarity)
                if(similarity < thresh):
                    cv2.putText(frame, person_name[:], \
                                (d.left(), d.top()-10), \
                                cv2.FONT_HERSHEY_SIMPLEX,1.2, \
                                (255, 255, 0), 3)
                    print(f'刷脸认证成功！！{person_name} 通过人脸识别，请开闸放行！！')
                    found = 1
                    break

            if(found == 0): # 识别失败
                cv2.putText(frame, 'unknown', (d.left(), d.top()-10), \
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, \
                            (255, 255, 0), 3)
                print('刷脸认证失败！！闸门关闭！！')
                break

    cv2.imshow('Face Recognition',frame)
    
    if cv2.waitKey(1) & 0xFF == 27: # ESC结束测试
        break
        
cap.release()
cv2.destroyAllWindows()

(2622,)
余弦距离 0.32654738426208496
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.32074403762817383
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.34030604362487793
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.345941960811615
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.25692516565322876
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.1257152557373047
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.13776344060897827
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.18217146396636963
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.17011749744415283
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.15461629629135132
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.23903542757034302
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.25089800357818604
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.3042909502983093
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.13686275482177734
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.3742554783821106
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.31246238946914673
刷脸认证成功！！120 通过人脸识别，请开闸放行！！
(2622,)
余弦距离 0.24421751499176025
刷脸认证成功！！120 

按ESC键结束测试。对测试结果满意吗？

## <font color ='red'> 祝贺您已经完成整个项目！！！</font> 

在人脸识别领域，已经初步打通任督二脉，可以通过实践更多的模型和方案作出比较分析。